#### Testing Pipeline

In [ ]:
test_image = "dataset/images/test"
test_csv = "dataset/test.csv"

test_df = pd.read_csv(test_csv)

class TestDataset(Dataset):
    def __init__(self, df, transform, dir_path):
        self.df = df.reset_index()
        self.transform = transform
        self.dir_path = dir_path


    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        img_path = self.df.loc[idx, 'filename']
        image = Image.open(os.path.join(self.dir_path, img_path)).convert("RGB")
        image = self.transform(image)
        return image

In [ ]:
test_data = TestDataset(test_df, transform = train_transform, dir_path = test_image)

In [ ]:
test_loader = DataLoader(test_data)

In [ ]:
pred = []

model.to(device)
with torch.no_grad():
    for images in tqdm(test_loader, desc="Testing"):
        images = images.to(device)
        outputs = model(images)
        _, preds = torch.max(outputs, 1)
        pred.append(int(preds.item()))




#### ROC-AUC Test

In [ ]:
from sklearn.metrics import roc_auc_score, roc_curve, accuracy_score, classification_report
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

# contoh data binary
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = RandomForestClassifier()
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
y_pred_proba = model.predict_proba(X_test)[:, 1]  # probabilitas kelas positif


In [ ]:
from sklearn.metrics import roc_auc_score, roc_curve
import torch
import numpy as np
import matplotlib.pyplot as plt

model.eval()
y_true, y_prob, y_pred = [], []

with torch.no_grad():
    for images, labels in val_loader:
        images, labels = images.to(device), labels.to(device)

        outputs = model(images)              # (batch, 1) jika pakai sigmoid
        probs = torch.sigmoid(outputs)       # pastikan dalam range [0,1]

        y_true.extend(labels.cpu().numpy())
        y_prob.extend(probs.cpu().numpy())

y_true = np.array(y_true).flatten()
y_prob = np.array(y_prob).flatten()

auc = roc_auc_score(y_true, y_prob)
print(f"ROC-AUC: {auc:.4f}")

# ROC curve
fpr, tpr, _ = roc_curve(y_true, y_prob)
plt.plot(fpr, tpr, label=f"AUC = {auc:.3f}")
plt.plot([0,1], [0,1], '--', color='gray')
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("ROC Curve")
plt.legend()
plt.show()
